In [1]:
train='/content/drive/MyDrive/train - train.csv'
test='/content/drive/MyDrive/test - test.csv'

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.corpus import stopwords
stops=stopwords.words('english')

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import regularizers

In [5]:
df_train=pd.read_csv(train)
df_test=pd.read_csv(test)

In [ ]:
df_train.head()

In [7]:
df_train['aspect_text']=df_train['aspect']+ ' | '+ df_train['text']

In [8]:
df_train.head()

,text,aspect,label,aspect_text
0,can you check whether its cancelled completely?,cancelled,1,cancelled | can you check whether its cancelle...
1,cannot rely on both milk delivery and grocery ...,Milk,0,Milk | cannot rely on both milk delivery and g...
2,"I get no notification, however the app is real...",notification,0,"notification | I get no notification, however ..."
3,"Love this app, but would love it even more if ...",view,1,"view | Love this app, but would love it even m..."
4,it does not let me load a clip on the scene,load,0,load | it does not let me load a clip on the s...


In [9]:
"removing stopwords"
new_texts=[]
for text in df_train['aspect_text'].tolist():
  char=[]
  for i in text.split():
    if i in stops:
      pass
    else:
      char.append(i)
  new_texts.append(' '. join(char))
  


In [10]:
df_train['new_texts']=new_texts

In [11]:
df_train.head(10)

,text,aspect,label,aspect_text,new_texts
0,can you check whether its cancelled completely?,cancelled,1,cancelled | can you check whether its cancelle...,cancelled | check whether cancelled completely?
1,cannot rely on both milk delivery and grocery ...,Milk,0,Milk | cannot rely on both milk delivery and g...,Milk | cannot rely milk delivery grocery items.
2,"I get no notification, however the app is real...",notification,0,"notification | I get no notification, however ...","notification | I get notification, however app..."
3,"Love this app, but would love it even more if ...",view,1,"view | Love this app, but would love it even m...","view | Love app, would love even Gantt charts ..."
4,it does not let me load a clip on the scene,load,0,load | it does not let me load a clip on the s...,load | let load clip scene
5,"i love notion as a tool, but the mobile just t...","tool,",2,"tool, | i love notion as a tool, but the mobil...","tool, | love notion tool, mobile takes way lon..."
6,unlimited yearly at $216/year,yearly,1,yearly | unlimited yearly at $216/year,yearly | unlimited yearly $216/year
7,"some times missing to deliver, some times diff...","deliver,",0,"deliver, | some times missing to deliver, some...","deliver, | times missing deliver, times differ..."
8,Would love the ability to “un-delete” deleted ...,tasks,1,tasks | Would love the ability to “un-delete” ...,tasks | Would love ability “un-delete” deleted...
9,please add this basic feature,Please,2,Please | please add this basic feature,Please | please add basic feature


DATA PRE-PROCESSING

In [14]:
num_words=6000
"Tokenizer remove punctuations and converts to lower_case"
tokenizer=Tokenizer(num_words=num_words,oov_token='oov', filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{}~\t\n')

In [15]:
tokenizer.fit_on_texts(df_train['new_texts'].values)

In [16]:
tok_texts=tokenizer.texts_to_sequences(df_train['new_texts'].values)

In [49]:
max_count=0
lenghts=[]
for i in tok_texts:
  x=len(i)
  lenghts.append(x)
lenghts=sorted(lenghts, reverse=True)


In [17]:
pad_texts=pad_sequences(tok_texts,maxlen=40, padding='post', truncating='post')

In [32]:
model=tf.keras.Sequential([
                           tf.keras.layers.Embedding(input_dim=num_words,output_dim=64,input_length=40),
                           tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16)),
                           tf.keras.layers.Dense(32, kernel_regularizer=regularizers.l2(0.001),  activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.Dense(32, kernel_regularizer=regularizers.l2(0.001), activation='relu'),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.Dense(3, activation='softmax')] )

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
label=df_train['label'].values

In [20]:
"Split train data to train and validation"
tr=pad_texts[:3501]
l_tr=label[:3501]
te=pad_texts[3501:]
l_te=label[3501:]


Training and Evaluating Model

In [33]:
model.fit(tr, l_tr, epochs=3)

Epoch 1/3
110/110 [==============================] - 7s 32ms/step - loss: 1.0802 - accuracy: 0.4444
Epoch 2/3
110/110 [==============================] - 3s 31ms/step - loss: 0.7305 - accuracy: 0.7132
Epoch 3/3
110/110 [==============================] - 4s 33ms/step - loss: 0.5141 - accuracy: 0.8258


In [35]:
model.evaluate(te, l_te)

16/16 [==============================] - 0s 6ms/step - loss: 0.9092 - accuracy: 0.6593


[0.9091842174530029, 0.6593186259269714]

In [41]:
x=np.argmax(model.predict(te), axis=1)

In [ ]:
x

In [45]:
from sklearn.metrics import classification_report
print(classification_report(l_te, x))

              precision    recall  f1-score   support

           0       0.70      0.65      0.67       214
           1       0.63      0.70      0.66       160
           2       0.63      0.62      0.63       125

    accuracy                           0.66       499
   macro avg       0.65      0.66      0.66       499
weighted avg       0.66      0.66      0.66       499

